### nibss settlement scheduler

In [0]:
print("Healthy")

In [0]:
import pandas as pd 
import numpy as np 
from sqlalchemy import create_engine
import psycopg2
import mysql.connector
import duckdb
import holidays as hols
from datetime import datetime, timedelta

import warnings
warnings.filterwarnings('ignore')

import json
with open('/Workspace/Credentials/db_data.json', 'r') as fp:
    data = json.load(fp)


Dumpdb = mysql.connector.connect(
  host=data['Dumpdb']['host'],
  user=data['Dumpdb']['user'],
  passwd=data['Dumpdb']['passwd'],
  database = data['Dumpdb']['database']
)



Mifosdb = mysql.connector.connect(
  host=data['mifos']['host'],
  user=data['mifos']['user'],
  passwd=data['mifos']['passwd'],
  database = data['mifos']['database']
)


vbizdb = mysql.connector.connect(
     host=data['vbizdb']['host'],
  user=data['vbizdb']['user'],
  passwd=data['vbizdb']['passwd'],
  database = data['vbizdb']['database']
)


host = data['redshift']['host']
user = data['redshift']['user']
passwd = data['redshift']['passwd']
database = data['redshift']['database']

conn = create_engine(f"postgresql+psycopg2://{user}:{passwd}@{host}:5439/{database}")
pd.set_option('display.float_format', lambda x: '%.2f' % x)


In [0]:
from datetime import datetime, timedelta
today = datetime.today().strftime('%Y-%m-%d')
yesterday =  (datetime.today() - timedelta(days = 4)).strftime('%Y-%m-%d')
last_7_days = (datetime.today() - timedelta(days = 7)).strftime('%Y-%m-%d')

now = datetime.today() #+ timedelta(hours=1)
now = now.strftime('%Y-%m-%d %H:%M:%S')

holidays = [holiday.strftime('%Y-%m-%d') for holiday in hols.Nigeria(years=[2022,2024])]

holidays.extend(['2022-12-27', '2023-10-02'])


print(today)
print(yesterday)
print(now)

In [0]:
username = data['auto-report']['email']
passwd = data['auto-report']['passwd']
print(username)

In [0]:
now = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
while (datetime.today().strftime('%Y-%m-%d') in holidays) or (datetime.today().weekday() >= 5):
    prev_day = (datetime.today() - timedelta(days = 1))
    
    while (prev_day.strftime('%Y-%m-%d') in holidays) or (prev_day.weekday() in [5,6]):
        
        prev_day = prev_day - timedelta(days = 1)
    
    else:
        prev_day = prev_day.strftime('%Y-%m-%d 14')
        
        notif = f'''SELECT
        concat(b.out_start_period, ' to ',  b.out_end_period) as outflow_cycle_period,
        concat(b.in_start_period, ' to ',  b.in_end_period) as inflow_cycle_period,

        b.out_volume,
        b.in_volume,
        b.out_value,
        b.in_value,
        b.out_value - b.in_value AS difference,
        1030000000 AS threshold,
        (1030000000 - (b.out_value - b.in_value)) AS threshold_balance,
        case when FORMAT((((1030000000 - (b.out_value - b.in_value)) / 1030000000) * 100), 3) < 1 
            then CONCAT(FORMAT((100 - (((1030000000 - (b.out_value - b.in_value)) / 1030000000) * 100)), 2) * -1, '%')
            ELSE concat(FORMAT((100 - (((1030000000 - (b.out_value - b.in_value)) / 1030000000) * 100)), 2), '%') end AS percent_consumption
        FROM 
        (SELECT
        min(nip.TRANSACTION_DATE) as in_start_period,
        max(nip.TRANSACTION_DATE) as in_end_period,
        CURDATE() AS DATE,
        COUNT(*) AS in_volume,
        a.out_start_period,
        a.out_end_period,
        a.out_volume,
        sum(nip.AMOUNT) AS in_value,
        a.out_value
        FROM TM_NIP_INWARD nip
        JOIN (SELECT
            CURDATE() AS DATE,
            min(ms.POSTING_DATE) as out_start_period,
            max(ms.POSTING_DATE) as out_end_period,
            COUNT(*) AS out_volume,
            sum(ms.AMOUNT) AS out_value
            FROM MM_BANK_SETTLEMENTS ms
            WHERE left(ms.POSTING_DATE, 13) >= '{prev_day}' -- and left(now(), 13)
            and ms.PARTNERS = 'NIPCREDIT'
            and ms.RESPONSE_CODE = '00') a ON a.date = CURDATE()
        WHERE left(nip.TRANSACTION_DATE, 13)  >= '{prev_day}' -- and left(now(), 13)
        and nip.TRANSACTION_STATUS = 'SUCCESS') b
        '''

        notif_df = pd.read_sql_query(notif, Dumpdb)
    
    break

# while (datetime.today().strftime('%Y-%m-%d') not in [holiday.strftime('%Y-%m-%d') for holiday in holidays.Nigeria(years=[2022,2023])]) or (datetime.today().weekday() < 5):
else:
    prev_day = (datetime.today() - timedelta(days = 1))
    
    while (prev_day.strftime('%Y-%m-%d') in holidays) or (prev_day.weekday() in [5,6]):
        
        prev_day = prev_day - timedelta(days = 1)
    
    else:
        if datetime.today().strftime('%H') < '14':
            prev_day = prev_day.strftime('%Y-%m-%d 14')

            notif = f'''SELECT
            concat(b.out_start_period, ' to ',  b.out_end_period) as outflow_cycle_period,
            concat(b.in_start_period, ' to ',  b.in_end_period) as inflow_cycle_period,

            b.out_volume,
            b.in_volume,
            b.out_value,
            b.in_value,
            b.out_value - b.in_value AS difference,
            1030000000 AS threshold,
            (1030000000 - (b.out_value - b.in_value)) AS threshold_balance,
            case when FORMAT((((1030000000 - (b.out_value - b.in_value)) / 1030000000) * 100), 3) < 1 
                then CONCAT(FORMAT((100 - (((1030000000 - (b.out_value - b.in_value)) / 1030000000) * 100)), 2) * -1, '%')
                ELSE concat(FORMAT((100 - (((1030000000 - (b.out_value - b.in_value)) / 1030000000) * 100)), 2), '%') end AS percent_consumption
            FROM 
            (SELECT
            min(nip.TRANSACTION_DATE) as in_start_period,
            max(nip.TRANSACTION_DATE) as in_end_period,
            CURDATE() AS DATE,
            COUNT(*) AS in_volume,
            a.out_start_period,
            a.out_end_period,
            a.out_volume,
            sum(nip.AMOUNT) AS in_value,
            a.out_value
            FROM TM_NIP_INWARD nip
            JOIN (SELECT
                CURDATE() AS DATE,
                min(ms.POSTING_DATE) as out_start_period,
                max(ms.POSTING_DATE) as out_end_period,
                COUNT(*) AS out_volume,
                sum(ms.AMOUNT) AS out_value
                FROM MM_BANK_SETTLEMENTS ms
                WHERE left(ms.POSTING_DATE, 13) >= '{prev_day}' -- and left(now(), 13)
                and ms.PARTNERS = 'NIPCREDIT'
                and ms.RESPONSE_CODE = '00') a ON a.date = CURDATE()
            WHERE left(nip.TRANSACTION_DATE, 13)  >= '{prev_day}' -- and left(now(), 13)
            and nip.TRANSACTION_STATUS = 'SUCCESS') b
            '''

            notif_df = pd.read_sql_query(notif, Dumpdb)

        else:

            new_cycle = datetime.today().strftime('%Y-%m-%d 14')

            notif = f'''SELECT
            concat(b.out_start_period, ' to ',  b.out_end_period) as outflow_cycle_period,
            concat(b.in_start_period, ' to ',  b.in_end_period) as inflow_cycle_period,

            b.out_volume,
            b.in_volume,
            b.out_value,
            b.in_value,
            b.out_value - b.in_value AS difference,
            1030000000 AS threshold,
            (1030000000 - (b.out_value - b.in_value)) AS threshold_balance,
            case when FORMAT((((1030000000 - (b.out_value - b.in_value)) / 1030000000) * 100), 3) < 1 
                then CONCAT(FORMAT((100 - (((1030000000 - (b.out_value - b.in_value)) / 1030000000) * 100)), 2) * -1, '%')
                ELSE concat(FORMAT((100 - (((1030000000 - (b.out_value - b.in_value)) / 1030000000) * 100)), 2), '%') end AS percent_consumption

            FROM 
            (SELECT
            min(nip.TRANSACTION_DATE) as in_start_period,
            max(nip.TRANSACTION_DATE) as in_end_period,
            CURDATE() AS DATE,
            COUNT(*) AS in_volume,
            a.out_start_period,
            a.out_end_period,
            a.out_volume,
            sum(nip.AMOUNT) AS in_value,
            a.out_value
            FROM TM_NIP_INWARD nip
            JOIN (SELECT
                CURDATE() AS DATE,
                min(ms.POSTING_DATE) as out_start_period,
                max(ms.POSTING_DATE) as out_end_period,
                COUNT(*) AS out_volume,
                sum(ms.AMOUNT) AS out_value
                FROM MM_BANK_SETTLEMENTS ms
                WHERE left(ms.POSTING_DATE, 13) >= '{new_cycle}' -- and left(now(), 13)
                and ms.PARTNERS = 'NIPCREDIT'
                and ms.RESPONSE_CODE = '00') a ON a.date = CURDATE()
            WHERE left(nip.TRANSACTION_DATE, 13)  >= '{new_cycle}' -- and left(now(), 13)
            and nip.TRANSACTION_STATUS = 'SUCCESS') b
            '''
            notif_df = pd.read_sql_query(notif, Dumpdb)
            


In [0]:
notif_df['in_volume'] = '{:,}'.format(notif_df['in_volume'][0])
notif_df['out_volume'] = '{:,}'.format(notif_df['out_volume'][0])
notif_df['in_value'] = 'N {:,.2f}'.format(notif_df['in_value'][0])
notif_df['out_value'] = 'N {:,.2f}'.format(notif_df['out_value'][0])
notif_df['difference'] = 'N {:,.2f}'.format(notif_df['difference'][0])
notif_df['threshold'] = 'N {:,.2f}'.format(notif_df['threshold'][0])
notif_df['threshold_balance'] = 'N {:,.2f}'.format(notif_df['threshold_balance'][0])

trans = notif_df.T
trans = trans.reset_index()
trans.rename(columns = {'index': 'Metrics', 0: 'Values'}, inplace = True)

thresh_p = float(trans['Values'][len(trans)-1].split('%')[0]) 

# if thresh_p < 1:
#     thresh_p = thresh_p * (-1)
# else:
#     thresh_p = thresh_p * 1

print(thresh_p)
print(trans)

In [0]:
if thresh_p >= 50:
    moment = datetime.today() #+ timedelta(hours=1)
    moment = moment.strftime('%Y-%m-%d %H:%M:%S')

    trans.to_csv('/Workspace/ReportDump/Settlement_Notifier/notif.csv', index =False)

    import csv
    from tabulate import tabulate
    from email.mime.multipart import MIMEMultipart
    from email.mime.text import MIMEText
    from email.mime.base import MIMEBase
    from email import encoders
    import smtplib

    me = 'auto-report@vfdtech.ng'
    #password = 'wmglfvqvnhdqgrrc'

    you = 'oladapo.omolaja@vfdtech.ng' 
    text = f"SETTLEMENT NOTIFICATION AS AT {moment}." + """
    Please see details below:
    {table}
    # """ 
    #th, td {{ padding: 5px; }}
    html = """
    <html>
    <style> 
    tab, th, td {{ border: 1px solid black; border-collapse: collapse; }}
    th, td {{ padding: 3px; }}
    </style>
    <body>
    <p>Dear Team, </br>
    </br>
    <p>Please be notified of NIBSS settlement details as at """+moment+""".
    </br>
    {tab}
    </br>
    <p>Regards,</p>
    </body>
    </html>
    """
    with open('/Workspace/ReportDump/Settlement_Notifier/notif.csv') as input_file:
        reader = csv.reader(input_file)
        data = list(reader)
    text = text.format(table=tabulate(data, headers="firstrow", tablefmt="grid"))
    html = html.format(tab=tabulate(data, headers="firstrow", tablefmt="html"))
    msg = MIMEMultipart(
    "alternative", None, [MIMEText(text), MIMEText(html,'html')])

    print(msg)
    msg['From'] = 'auto-report@vfdtech.ng'
    #msg['To'] = 'Oladapo.Omolja@vfdtech.ng'
    msg['To'] = 'treasurynotification@vfd-mfb.com, olugbenga.paseda@vfdtech.ng, victor.nwaka@vfdtech.ng, daniel.babatunde@vfdtech.ng, olawale.kareem@vfdtech.ng, oluwasijibomi.bamgboye@vfdtech.ng, ikechukwu.mbaliri@vfd-mfb.com, bukola.adedoyin@vfd-mfb.com, samuel.adegbuyi@vfd-mfb.com, ngozi.nwagbo@vfd-mfb.com'  #'AppSupport@vfdtech.ng'
    msg['Bcc'] = 'data-team@vfdtech.ng'
    msg['Subject'] = f'SETTLEMENT NOTIFICATION AS AT {moment}'

    ## ATTACHMENT PART OF THE CODE IS HERE
    # attachment = open(SourcePathName, 'rb')
    # part = MIMEBase('application', "octet-stream")
    # part.set_payload((attachment).read())
    # encoders.encode_base64(part)
    # part.add_header('Content-Disposition', "attachment; filename= %s" % filename)
    # msg.attach(part)
    server = smtplib.SMTP('smtp-mail.outlook.com', 587)
    server.ehlo()
    server.starttls()
    server.login(username, passwd)
    server.send_message(msg)
    server.quit()

else:

    print('Threshold consumption still within a good zone!!!')

